https://habrahabr.ru/post/314490/

conda install tensorflow

pip install git+git://github.com/Theano/Theano.git

pip install git+git://github.com/fchollet/keras.git

conda config --add channels menpo

conda install opencv

In [1]:
%matplotlib inline
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np
import os
import h5py
from matplotlib import pyplot as plt

import theano
theano.config.openmp = True

Using Theano backend.


In [2]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(1000, activation='softmax'))


    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')
    return model


model = VGG_16('vgg16_weights.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

Model loaded.


In [3]:
path = "./images/"
ims = []
files = []
for f in os.listdir(path):
    print f
    if (f.endswith(".jpg")) and (os.stat(path+f) > 10000):
        try:
            files.append(f.encode('utf-8'))
            im = cv2.resize(cv2.imread(path+f), (224, 224)).astype(np.float32)
    #         plt.imshow(im)
    #         plt.show()
            im[:,:,0] -= 103.939
            im[:,:,1] -= 116.779
            im[:,:,2] -= 123.68
            im = im.transpose((2,0,1))
            im = np.expand_dims(im, axis=0)
            ims.append(im)
        except:
            print f

images = np.vstack(ims)
print images.shape

1.jpg
1hLF3Lc85H4.jpg
1j9sKc5zceQ.jpg
5e0rMflOBsY.jpg
7GAfCT7A6ws.jpg
9zMOyDvCfCY.jpg
EJfOOVm6DmQ.jpg
gzgWIG-YkDA.jpg
h9C3QzpaZa8.jpg
MS-sSVj1y1I.jpg
oE6boCQrzU8.jpg
O-y-9g0MGXA.jpg
ya.jpg
111111111.jpg
aDjSG0XuxSU.jpg
rcnqQDIJJYo.jpg
pBY-D1YgJ1g.jpg
76ImYigyHm0.jpg
lP_Cw4CUvQ8.jpg
8prZCBF-OrY.jpg
PMtOdnRLbLY.jpg
7xCWhuhc1MY.jpg
J7hbk8ziRrI.jpg
NXHhp4c15tU.jpg
(24, 3, 224, 224)


In [ ]:
# r1 =[]
# r2= []
# for i,x in enumerate(files):
#     r1.append(((x[:-4]),i))
#     r2.append((i,(x[:-4])))
# extid_to_intid_dict = dict(r1)
# intid_to_extid_dict = dict(r2)
# print extid_to_intid_dict
# print intid_to_extid_dict

In [4]:
out = model.predict(images)
print out
print out.shape

[[ 0.43810225  0.          0.         ...,  0.          0.          0.        ]
 [ 2.22679138  0.          0.         ...,  0.          5.31461573
   0.03604525]
 [ 1.50524127  0.83813     0.         ...,  0.          1.77447152
   0.09459919]
 ..., 
 [ 0.15372232  0.          0.         ...,  0.          1.46440911
   2.31832743]
 [ 0.34775668  0.          0.         ...,  0.          2.39366674
   1.74745083]
 [ 0.          0.          0.         ...,  0.          0.          0.07502443]]
(24, 4096)


In [ ]:
import h5py
h5f = h5py.File('data.h5', 'w')
h5f.create_dataset('dataset_1', data=out)
h5f.close()

In [ ]:
np.save('data', out)

https://github.com/manahl/arctic

In [5]:
from sklearn.metrics.pairwise import pairwise_distances
extid = 'PMtOdnRLbLY'
i = extid_to_intid_dict[str(extid)]
print i
plt.imshow(cv2.imread(path+files[i]))
plt.show()
dist = pairwise_distances(out[i],out, metric='euclidean', n_jobs=1)

top = np.argsort(dist[0])[0:7]

for t in top:
    print t,dist[0][t]
    plt.imshow(cv2.imread(path+files[t]))
    plt.show()

NameError: name 'extid_to_intid_dict' is not defined

In [ ]:
import joblib
joblib.dump((extid_to_intid_dict,intid_to_extid_dict,out),"wo_1_layer.pkl")